<a href="https://colab.research.google.com/github/impos0108/AI_basic_for_atmospheric_science/blob/main/MLP_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 139921017.05it/s]


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 33583502.58it/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 43236751.79it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 22151777.64it/s]


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [5]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.2831
Epoch [1/5], Step [200/600], Loss: 0.2102
Epoch [1/5], Step [300/600], Loss: 0.1765
Epoch [1/5], Step [400/600], Loss: 0.1418
Epoch [1/5], Step [500/600], Loss: 0.1109
Epoch [1/5], Step [600/600], Loss: 0.1150
Epoch [2/5], Step [100/600], Loss: 0.1494
Epoch [2/5], Step [200/600], Loss: 0.1114
Epoch [2/5], Step [300/600], Loss: 0.1274
Epoch [2/5], Step [400/600], Loss: 0.1142
Epoch [2/5], Step [500/600], Loss: 0.1369
Epoch [2/5], Step [600/600], Loss: 0.1575
Epoch [3/5], Step [100/600], Loss: 0.0892
Epoch [3/5], Step [200/600], Loss: 0.0826
Epoch [3/5], Step [300/600], Loss: 0.0748
Epoch [3/5], Step [400/600], Loss: 0.0613
Epoch [3/5], Step [500/600], Loss: 0.1453
Epoch [3/5], Step [600/600], Loss: 0.0810
Epoch [4/5], Step [100/600], Loss: 0.0168
Epoch [4/5], Step [200/600], Loss: 0.0295
Epoch [4/5], Step [300/600], Loss: 0.0593
Epoch [4/5], Step [400/600], Loss: 0.0400
Epoch [4/5], Step [500/600], Loss: 0.0130
Epoch [4/5], Step [600/600], Loss:

In [9]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 97.98 %
